In [1]:
import csv
import datetime 
import re

f=open("有课互联签到原始数据.csv",encoding='utf-8', mode = 'r')
lines=f.readlines()
csvrows=csv.reader(lines,delimiter=',')
startpattern='^(^|上午|下午|:)(\d+)(:|点|：)(\d+|到|~|-)'
endpattern='(到|~|-)(\d+)(:|点|：)(\d+|$)(\ +|$)$'
dianpingpattern='(^|.+)(点评:|点评：|特点:|特点：|点评：|特点：|总结经验:|评价:|评价：)(.+|$)'
mxwarning=""

#从签到备注中删除补签正常签等域,如果过短，根据有课互联前10列数据猜测补充
def getQiandaobeizhu(ykvols,qiandao_org):    
    global mxwarning
    qcols=qiandao_org.split("|")
    qdbz=""
    for qcol in qcols:
        #删除 补签 正常签 以及首个空字符
        if qcol == "补签" or qcol == "正常签" or qcol=="" or qcol=="\n": 
            continue
        else:
            qdbz+=qcol
            qdbz+="|"
 #   print(qdbz)
    qcols=qdbz.split("|")
    #如果缺乏上课地址以及上课时间信息，从有课互联记录里恢复
     #只有姓名和签到时间
 #   print(qcols)
 #   print(len(qcols))
    if len(qcols)==3:
        mxwarning+="记录不全自动补齐需要复查，"
        #添加上课地址为有课互联校区地址
        qdbz+=ykvols[9]
        qdbz+="|"
        #添加上课时间为零点开始的有课互联销课时间
        qdbz+="00:00"
        qdbz+="-"
        qdbz+=str(int(float(ykvols[5])))+":00"
        qdbz+="|"
        qdbz+="学习内容：1，2，3 特点：|"
    #只有姓名和签到时间和签到地点
    if len(qcols)==4:
        mxwarning+="记录不全自动补齐需要复查，"
        #添加上课时间为零点开始的有课互联销课时间
        qdbz+="00:00"
        qdbz+="-"
        qdbz+=str(int(float(ykvols[5])))+":00"
        qdbz+="|"
        qdbz+="学习内容：1，2，3 特点：|"
  #  print(qdbz)
    return qdbz

#从签到备注中获得教师和助教信息
def getTeacherAndTA(qiandaobeizhu):
    global mxwarning
    #    print(qiandaobeizhu)
    qcols=qiandaobeizhu.split("|")
#    print(qcols)
    if len(qcols) <2:
        return 'NA','NA'    
    tryteacher=qcols[0]
    TA="NA"
    #老师名字长度大于4，说明有助教，填上助教信息,主课老师位置排第一,第二个为助教，第三个不计入系统
    if(len(tryteacher)<4):
        teacher=tryteacher
    else:
        teas=tryteacher.split("、")
        teacher=teas[0]  
        if len(teas)>1:
            TA=teas[1]
        if len(teas)>2:
            mxwarning+="超过一个助教不计入系统，"
    return teacher,TA

#从签到备注中获得上课日期信息
def getTeachDate(qiandaobeizhu):
    global mxwarning
    counter=0;
#    print(qiandaobeizhu)
    qcols=qiandaobeizhu.split("|")
#    print(qcols)
    objDate = datetime.datetime.strptime("2018-01-01","%Y-%m-%d")
    teachdatestr=objDate.strftime("%Y-%m-%d")

    if len(qcols) <2:
        return 'NA' 
    #查找签到备注中每一个字段看有没有年月日时间信息，有的话计算为上课日期
    for qcol in qcols:
        counter+=1
        try:
            #找到了则返回日期字符串，同时返回日期的位置
 #           print(qcol)
            #删除少量的时间后面带回车的情况比如鲁运龙后面
            qcolnns0=qcol.split("\n")
            qcolnns=qcolnns0[0].split("少")
            objDate = datetime.datetime.strptime(qcolnns[0],"%Y-%m-%d")
            teachdatestr=objDate.strftime("%Y-%m-%d")
 #           print(counter)
            return teachdatestr,counter-1
        except:
#            print("datenotfound")
             continue
     #没有找到，返回默认值，日期位置设为100
    mxwarning+="没找到日期严重错误，"
    return teachdatestr,100

#获得上课地点，去除补签正常签等信息后，上课地点永远在上课日期之后
def getTeachAddr(qiandaobeizhu):
    global mxwarning
    teachaddr="NA"
    qcols=qiandaobeizhu.split("|")
#    print(qcols)
    if len(qcols) <2:
        return 'NA'  
    #先找上课日期，得到上课日期的位置
    teachdate,pos=getTeachDate(qiandaobeizhu)
    if pos >5:
        return 'NA'
 #   print(pos)
    teachaddr=qcols[pos+1]
    return teachaddr

#获得上课时间
def getTeachTime(qiandaobeizhu):
    global mxwarning
    #先找上课日期，得到上课日期的位置
    teachdate,pos=getTeachDate(qiandaobeizhu)
    qcols=qiandaobeizhu.split("|")
    #正常情况下日期后面的字段是地点，再后面是上课时间段，因此pos+2
#   print(qcols)
    #看能否找到时间模式，不能则选紧接着的字段作为上课时间段字段
    trytimestr=qcols[pos+2]
    gsearchstart=re.search(startpattern,trytimestr)
    if gsearchstart is None:
        tpos=pos+1
        mxwarning+="地点和日期顺序错误，"
    else:
        tpos=pos+2
 #   print(tpos)
    ttimestr=qcols[tpos]
    #   ttimestr="16:00~18:00,19:00~20:00"
    timestrs=ttimestr.split(",")
    teachtime=""
    for timestr in timestrs:
        #timestr 16:00~18:00
  #      print(timestr)
        teachtimesegstart="23:00"
        teachtimesegend="24:00"
        gsearchstart=re.search(startpattern,timestr)
        gstart=gsearchstart.groups()
        gsearchend=re.search(endpattern,timestr)
        if gsearchend is None:
            gsearchend=gsearchstart
        gend=gsearchend.groups()
        
        if not gstart[3].isdigit():
            startm='0'
        else:
            startm=gstart[3]
  
    
        if not gend[3].isdigit():
            endm='0'
        else:
            endm=gend[3]

   #     print(gstart)
   #     print(gend)
        
        sgstart=gstart[1]+":"+startm
        sgend=gend[1]+":"+endm

   #     print(sgstart)
   #     print(sgend)
        objDate = datetime.datetime.strptime(sgstart,"%H:%M")
        teachtimesegstart=objDate.strftime("%H:%M")
        objDate = datetime.datetime.strptime(sgend,"%H:%M")
        teachtimesegend=objDate.strftime("%H:%M")
       
        teachtime+=teachtimesegstart+"-"+teachtimesegend+"*"
    return teachtime

#获得上课内容
def getTeachCont(qiandaobeizhu):
    global mxwarning
    qdbz=qiandaobeizhu.split("|")
    qdbztc=qdbz[4]
#    print(qdbztc)
    qdzs=qdbztc.split("\n")
    teachc=""
    for qdz in qdzs:        
        gs=re.search(dianpingpattern,qdz)
#        print("!!!!!!!!!!!!!!!!!!!qdz!!!!!!!!!!!!!!!!!")
#        print(qdz)
        if gs is None:
            teachc+=qdz
        else:
            gsg=gs.groups()
#            print("!!!!!!!!!!!!!!for gsg!!!!!!!!!!!!!!!!!!")
#            print(gsg)
            teachc+=gsg[0]
    return teachc
  
#获得学生评价
def getStuComment(qiandaobeizhu):
    global mxwarning
    qdbz=qiandaobeizhu.split("|")
    qdbztc=qdbz[4]
#    print(qdbztc)
    qdzs=qdbztc.split("\n")
    teachc=""
    stucomment=""
    for qdz in qdzs:        
        gs=re.search(dianpingpattern,qdz)
#        print("!!!!!!!!!!!!!!!!!!!qdz!!!!!!!!!!!!!!!!!")
#        print(qdz)
        if gs is None:
            teachc+=qdz
        else:
            gsg=gs.groups()
#            print("!!!!!!!!!!!!!!for gsg!!!!!!!!!!!!!!!!!!")
#            print(gsg)
            stucomment+=gsg[2]
            continue
        if stucomment != "":
            stucomment+=qdz 
    return stucomment
    
#获得其它信息
def getOtherComments(qiandaobeizhu):
    global mxwarning
    othercomments=mxwarning
    return othercomments

#主程序从这里开始
counter=0
lenrow=0
rows=[]

#将签到备注转换为表格中的11列~17列
for row in csvrows:
    newrow=row[0:11]
    if(counter==0):
        newrow[10:18]=["授课教师","助教","授课日期","授课地点","授课时间","授课内容","学生点评","其它备注"]
    else:
        newrow[10:18]=["UNKNOWN_TEACHER","UNKNOWN_TA","UNKOWN_DATE","UNKONW_ADDR","UNKOWN_TIME","UNKONW_CONT","UNKONW_STUCOMENTS","OTHERS"]
        #原始签到备注
        mxwarning=""
        qiandao_org=row[11]
        #删除 补签 正常签等标记
        qiandaobeizhu=getQiandaobeizhu(row[0:10],qiandao_org)
        print(qiandaobeizhu)
        #得到教师和助教信息
        (teacher,TA)=getTeacherAndTA(qiandaobeizhu)
        newrow[10:12]=[teacher,TA]
        #得到授课日期
        datestring,counter=getTeachDate(qiandaobeizhu)
        newrow[12]=datestring
        #得到授课地点
        teachaddr=getTeachAddr(qiandaobeizhu)
        newrow[13]=teachaddr
        #得到授课时间
        timestring=getTeachTime(qiandaobeizhu)
        newrow[14]=timestring
        #得到授课内容
        teachcont=getTeachCont(qiandaobeizhu)
        newrow[15]=teachcont
        #获得学生评价
        stucomment=getStuComment(qiandaobeizhu)
        newrow[16]=stucomment
        #获得其它信息        
        othercomment=getOtherComments(qiandaobeizhu)
        newrow[17]=othercomment
    print(newrow)
    print('\n')
    print('\n')
    rows.append(newrow)
    lenrow+=len(row)
    counter+=1
    
print("总共",counter,"行，总共",lenrow,"列,每行",lenrow/counter,"列")
print(rows[0:4])

fw=open("有课互联签到加工数据.csv",'w',encoding='utf8',newline='')
csvwriter=csv.writer(fw)
csvwriter.writerows(rows)
fw.close()



['\ufeff签到编号', '签到日期', '签到时间', '学生姓名', '课程名称', '课时消耗', '剩余课时', '总课时', '签到操作人', '校区', '授课教师', '助教', '授课日期', '授课地点', '授课时间', '授课内容', '学生点评', '其它备注']




谢博|2018-06-16|漕东支路95号|00:00-2:00|学习内容：1，2，3 特点：|
['768', '2018-07-18', '19:55', '朱凯文', '3D打印和电子积木', '2.00', '41.00', '60.00', '前台老师', '少创派', '谢博', 'NA', '2018-06-16', '漕东支路95号', '00:00-02:00*', '学习内容：1，2，3 ', '', '记录不全自动补齐需要复查，']




鲁运龙|2018-04-21|国定东路200号|9:00-16:00|巡线|
['767', '2018-07-18', '19:50', '沈柏安', 'Robotex赛前培训小车巡线', '6.00', '0.00', '18.00', '前台老师', '少创派', '鲁运龙', 'NA', '2018-04-21', '国定东路200号', '09:00-16:00*', '巡线', '', '']




鲁运龙|2018-04-14|国定东路200号|9:00-16:00|巡线|
['766', '2018-07-18', '19:50', '沈柏安', 'Robotex赛前培训小车巡线', '6.00', '6.00', '18.00', '前台老师', '少创派', '鲁运龙', 'NA', '2018-04-14', '国定东路200号', '09:00-16:00*', '巡线', '', '']




鲁运龙|2018-04-07|国定东路200号|9:00-16:00|巡线|
['765', '2018-07-18', '19:50', '沈柏安', 'Robotex赛前培训小车巡线', '6.00', '12.00', '18.00', '前台老师', '少创派', '鲁运龙', 'NA', '2018-04-07', '国定东路200号', '09:00-16:00*', '巡线',

['741', '2018-07-18', '18:36', '茅轩铭', '3D打印机组装及建模', '2.00', '19.00', '30.00', '唐沁钰', '漕东路', '阮煜东', 'NA', '2018-07-07', '漕东支路95号', '16:45-18:20*', '按照网上的图片，动手进行了我的世界中苦力怕的建模，在其中使用和复习了对齐等命令，并且使用了一些数学计算 ', '', '']




阮煜东|2018-06-30|漕东支路95号|16:45-18:20|按照课件，在老师的演示下一步步建立了一个抽屉的模型，其中为他讲解了抽屉建模需要注意的空间原理，对培养孩子的空间能力是有好处的 |
['740', '2018-07-18', '18:35', '茅轩铭', '3D打印机组装及建模', '2.00', '21.00', '30.00', '唐沁钰', '漕东路', '阮煜东', 'NA', '2018-06-30', '漕东支路95号', '16:45-18:20*', '按照课件，在老师的演示下一步步建立了一个抽屉的模型，其中为他讲解了抽屉建模需要注意的空间原理，对培养孩子的空间能力是有好处的 ', '', '']




阮煜东|2018-06-23|漕东支路95号|16:45-18:20|1.将自主设计的地钻进行改良，使用了对齐等功能加强了作品的整体感和平衡性 2.安装了ripeterhost切片软件，讲授了导出模型后切片的方法，从此之后可以进行自主打印 评价:学生具有很强的好奇心和动手能力，对建模和设计自己的作品充满了热情 |
['739', '2018-07-18', '18:35', '茅轩铭', '3D打印机组装及建模', '2.00', '23.00', '30.00', '唐沁钰', '漕东路', '阮煜东', 'NA', '2018-06-23', '漕东支路95号', '16:45-18:20*', '1.将自主设计的地钻进行改良，使用了对齐等功能加强了作品的整体感和平衡性 2.安装了ripeterhost切片软件，讲授了导出模型后切片的方法，从此之后可以进行自主打印 ', '学生具有很强的好奇心和动手能力，对建模和设计自己的作品充满了热情 ', '']




阮煜东|2018-06-17|漕东支路95号|

In [108]:
import re
import datetime 

timestr="学习内容：1，2，3 特点："
gsearchstart=re.search("^(.+)(特点：)",timestr)
print(gsearchstart)
gstart=gsearchstart.groups()
print(gstart)

<_sre.SRE_Match object; span=(0, 14), match='学习内容：1，2，3 特点：'>
('学习内容：1，2，3 ', '特点：')
